In [73]:
import warnings
warnings.simplefilter("ignore")

## Escalamiento de los datos

In [74]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Diversos algoritmos son sensibles a la escala en la que viene cada feature. **Re-escalarlos** puede traer significativas mejoras de rendimiento.


Existen distintas estrategias de escalamiento de tus features, pero la más común es la estandarización donde convertimos la variable para que la distribución de esta siga una distribución que es Gaussiana de media 0 y de desviación estandar 1.

In [75]:
from sklearn.model_selection import train_test_split

X = pd.read_csv('./documents/X.csv')
Y = X['worldwide_gross']
X = X.drop('worldwide_gross',axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [76]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [77]:
scaler.mean_

array([3.27037470e+07, 3.02575227e+05, 5.48701017e+05, 3.63775813e+05,
       1.00721485e+04, 3.84731499e+07, 6.45506823e+00])

In [78]:
scaler.scale_

array([4.11817844e+07, 1.08403755e+07, 1.27506161e+07, 1.25184077e+07,
       1.63357430e+04, 2.28607730e+08, 1.07833770e+00])

In [79]:
scaler.transform(X_train)

array([[-0.50274041, -0.02772729, -0.04303311, ..., -0.17790121,
        -0.11580164,  0.50534427],
       [-0.78806073, -0.02772738, -0.04303315, ..., -0.53221629,
        -0.16719973,  0.13440296],
       [-0.7577075 , -0.0277272 , -0.04303315, ..., -0.37776968,
        -0.16173184,  0.04166763],
       ...,
       [-0.06565395, -0.02772618, -0.04303311, ..., -0.51599419,
         0.02855043, -0.0510677 ],
       [ 5.8835783 , -0.02772618, -0.04303311, ...,  2.18446456,
         0.77218233,  0.04166763],
       [-0.76742053, -0.0277272 , -0.04303319, ..., -0.61657119,
        -0.16741976, -1.62756827]])

In [80]:
X_train_scaled, X_test_scaled = (scaler.transform(X_train), scaler.transform(X_test))

In [81]:
from sklearn.linear_model import Lasso

model = Lasso()
model_scaled = Lasso()

model.fit(X_train, Y_train)
model_scaled.fit(X_train_scaled, Y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [82]:
print('Normal model: ', model.score(X_test, Y_test))
print('Scaled model: ', model_scaled.score(X_test_scaled, Y_test))

Normal model:  0.5245528162032734
Scaled model:  0.5245528173720726


Los modelos de regresión no se ven afectados por el escalamiento de las features. Los de clasificación sí.

## Simplificar las transformaciones con pipelines

Para hacer tu código más reproducible, y para evitar tener que aplicar multiples veces una misma transformación te recomendamos utilizar `sklearn.pipeline.make_pipeline` que permite encadenar transformaciones a tus modelos.

In [83]:
from sklearn.pipeline import make_pipeline

model_scaled = make_pipeline(StandardScaler(), Lasso())
model_scaled.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('lasso',
                 Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=1000, normalize=False, positive=False,
                       precompute=False, random_state=None, selection='cyclic',
                       tol=0.0001, warm_start=False))],
         verbose=False)

## Crear nuevas features de forma automatica

In [84]:
A = np.arange(6).reshape(3, 2)
A

array([[0, 1],
       [2, 3],
       [4, 5]])

In [85]:
from sklearn.preprocessing import PolynomialFeatures

transformer = PolynomialFeatures(2)
transformer.fit_transform(A)

array([[ 1.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  2.,  3.,  4.,  6.,  9.],
       [ 1.,  4.,  5., 16., 20., 25.]])


PolynomialFeatures transforma una matriz ![](./documents/math1.svg)

In [86]:
X.shape

(4104, 7)

In [87]:
transformer = PolynomialFeatures(2)
transformer.fit_transform(X).shape

(4104, 36)

In [88]:
model_poly = make_pipeline(PolynomialFeatures(2), Lasso())
model_poly.fit(X_train, Y_train)
model_poly.score(X_test, Y_test)

0.5948644583973391

## Crear features categóricas

En terminos de Machine Learning a las features que pueden tomar un número finito de valores se les llama categóricas. Ejemplos para esto són: género, páis, grado académico, etc.

Un mapeo del tipo $\{Perú, Chile, Colombia, Venezuela\} \rightarrow \{1, 2, 3, 4\}$ tiene el problema de asignarle un ordén a los valores posibles de la categoría. Este orden impacta de distintas maneras los algoritmos de Machine Learning, por ejemplo aquellos que dependen de la topología de $R^n$ y de la función de distancia entre puntos en este espacio, considerarán que ciertas categorías se encuentran más cercanas unas de otras, siendo que esto es generado puramente artificialmente por el encoding, y no por las datos per se.

Para no introducir información falsa o erronéa en nuestro modelos existen formas más inteligentes de encodear nuestros datos.

### Encoding one-hot

Este encoding consiste en asignarle una columna a cada categoría y rellenarla con 0 y 1 de la forma siguiente:

In [89]:
d = pd.DataFrame([['Chile','Colombia','Colombia','Venezuela'],['hombre','mujer','hombre','mujer']])
d = d.T
d.columns = pd.Index(['pais','genero'])
d

,pais,genero
0,Chile,hombre
1,Colombia,mujer
2,Colombia,hombre
3,Venezuela,mujer


In [90]:
pd.get_dummies(d)

,pais_Chile,pais_Colombia,pais_Venezuela,genero_hombre,genero_mujer
0,1,0,0,1,0
1,0,1,0,0,1
2,0,1,0,1,0
3,0,0,1,0,1


Sklearn también ofrece un objeto OneHotEncoder pero es un poco más díficil de utilizar, así que por criterios pedagogicos hemos elegido pd.get_dummies. Sin embargo el objeto de sklearn tiene la ventaja de ser pipeable, por lo que es bueno considerarlo para ciertos casos de uso.

Cuantas columnas generaríamos con un one-hot encoding de nuestras features categóricas?

In [91]:
movies_obj = pd.read_csv('./documents/movies_obj.csv')

In [92]:
movies_obj.apply(pd.Series.nunique).sort_values()

color                2
content_rating      18
language            47
country             65
genres             914
actor_1_name      2097
director_name     2398
actor_2_name      3032
actor_3_name      3521
plot_keywords     4760
movie_title       4917
dtype: int64

Las features más informativas son las del casting. Si embargo haciendo un one-hot encoding de estas estaríamos aumentando la dimensión por 2000 y algo!!

## Encoding Binario

Esta técnica no es canónica por lo que tendremos que buscarla en otra librería. Sin embargo el autor tuvo la buena idea de hacer su API compatible con la de sklearn, así que no tendremos ninguna dificultad en usarla.

$$ Categoria \rightarrow Numero \rightarrow Binario \rightarrow Columnas $$

In [93]:
!pip install category_encoders

You should consider upgrading via the 'a:\anaconda3\python.exe -m pip install --upgrade pip' command.


In [106]:
categoricals = pd.read_csv('./documents/categoricals.csv').set_index('Unnamed: 0')

In [107]:
categoricals.head(2)

,actor_1_name,director_name
Unnamed: 0,,
0,CCH Pounder,James Cameron
1,Doug Walker,Doug Walker


In [108]:
categoricals = categoricals.reset_index(drop=True).fillna(0)

In [109]:
X_binenc = pd.concat([X,categoricals],axis=1)

In [110]:
X_binenc.head(2)

,production_budget,title_year,aspect_ratio,duration.1,cast_total_facebook_likes,budget,imdb_score,actor_1_name,director_name
0,425000000.0,2.009000e+03,1.780000e+00,1.780000e+02,4834.0,2.370000e+08,7.9,CCH Pounder,James Cameron
1,306000000.0,5.911656e+08,5.911656e+08,5.911656e+08,143.0,5.911656e+08,7.1,Doug Walker,Doug Walker


In [111]:
import category_encoders as ce

encoder = ce.BinaryEncoder(cols=['actor_1_name','director_name'])

In [112]:
encoder.fit_transform(X_binenc).shape

(4104, 31)

In [113]:
X_binenc = encoder.fit_transform(X_binenc)

In [114]:
Xb_train, Xb_test, Y_train, Y_test = train_test_split(X_binenc,Y)

In [115]:
X_train, X_test = (Xb_train[X.columns],Xb_test[X.columns])

In [116]:
model_binenc = Lasso()
model = Lasso()

In [118]:
model_binenc.fit(Xb_train,Y_train)
model.fit(X_train,Y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [120]:
print(model_binenc.score(Xb_test,Y_test))
print(model.score(X_test,Y_test))

0.6265574556127946
0.6207020120171811


Aumentamos el rendimiento de nuestro algoritmo pero no de forma significativa. Mantengamos entonces la dimensionalidad de nuestro espacio de features baja, y vamos a buscar modelos más complejos.

## Conocimiento experto

Una grán parte del diseño de las features pasa por un **conocimiento espécifico del dominio en el que se esta trabajando**.
Por ejemplo para analizar una imagen nuestro cerebro no se concentra en los millones de pixeles de una imagen, pero sólo en algunos relevantes como los de los contornos. Durante un buen tiempo **los sistemas de visión de computadores encodeaban features que traducían este conocimiento experto (contornos).**
Una de las únicas formas de obtener este conocimiento de forma sistemática es ir a bucear en repositorios de papers de Machine Learning como Arxiv, y estudiar la investigación que se ha hecho sobre el dominio específico.

## Más datos de calidad

Nada le gana conseguir más datos que sean encodeables en features de calidad.

### Píramide de Maslow del Machine Learning

![](./documents/maslow.png)

Contamos con la base de datos de ganancias de las péliculas el primer fin de semana de exhibición, así como la cantidad de cines en la que fue estrenada.